# Цены компаний и объем

In [157]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from sqlalchemy import create_engine 
import requests
import datetime
import time
from random import uniform, randint

### Загрузка данных с базы данных

In [3]:
username = '...'
password = '...'
host = '...'
port = '...'
database = '...'
conn_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

In [65]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

companies_df = pd.read_sql_query('SELECT * FROM companies_tickets', conn)
conn.close()

### Загрузка данных с Мосбиржи и обогащение существующего датасета

In [24]:
companies_string = 'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{}/candles.json?from={}&till={}&interval=1'

In [ ]:
companies_df['price_release'] = np.nan
companies_df['volume_release'] = np.nan
companies_df['price_lag_30'] = np.nan
companies_df['volume_lag_30'] = np.nan

In [ ]:
companies_df = companies_df.sort_values(by=['datetime'])

In [94]:
for i in tqdm(range(len(companies_df))):
    start = companies_df['datetime'][i]
    end = companies_df['datetime'][i] + datetime.timedelta(minutes=30)
    company = companies_df['company'][i]
    connect_timeout = 300
    read_timeout = 120
    n = requests.get(companies_string.format(company, start, end), timeout=(connect_timeout, read_timeout)).json()
    try:
        companies_df.loc[i, 'price_release'] = n['candles']['data'][0][0]
        companies_df.loc[i, 'volume_release'] = n['candles']['data'][0][4]
        companies_df.loc[i, 'price_lag_30'] = n['candles']['data'][-1][0]
        companies_df.loc[i, 'volume_lag_30'] = n['candles']['data'][-1][4]
    except IndexError:
        continue
    #if i % randint() == 0:
    #   delay = uniform(5, 10)
    #    time.sleep(delay)


100%|██████████| 7542/7542 [35:07<00:00,  3.58it/s]  


In [193]:
companies_df = pd.read_csv('companies_df.csv', index_col=0)

In [194]:
companies_df.isna().sum()

website             0
section             0
url                 0
header              0
body                0
tags              733
body_length         0
datetime            0
company             0
price_release     275
price_lag_30      275
volume_release    275
volume_lag_30     275
dtype: int64

In [195]:
companies_df = companies_df.dropna(subset=['price_release'])

### Получение разниц

In [196]:
companies_df['price_diff'] = companies_df['price_lag_30'] - companies_df['price_release']
companies_df['price_diff_percent'] = companies_df['price_diff'] / companies_df['price_release'] * 100
companies_df['volume_diff'] = companies_df['volume_lag_30'] - companies_df['volume_release']
companies_df['volume_diff_percent'] = companies_df['volume_diff'] / companies_df['volume_release'] * 100

In [197]:
companies_df

,website,section,url,header,body,tags,body_length,datetime,company,price_release,price_lag_30,volume_release,volume_lag_30,price_diff,price_diff_percent,volume_diff,volume_diff_percent
0,РИА,Экономика,https://ria.ru/20230103/banki-1842907991.html,Эксперт рассказал о развитии исламского банкин...,Порядка 10-15 филиалов исламского банкинга м...,"['Экономика', 'Россия', 'Сбербанк России']",1759,2023-01-03 10:16:00,SBER,142.75,142.62,11917688.40,2958789.90,-0.13,-0.091068,-8958898.5,-75.173122
1,РИА,Экономика,https://ria.ru/20230103/gazprom-1842940413.html,"Экспорт ""Газпрома"" снизился до минимума с конц...","Объём экспорта ""Газпрома"" в прошедшем году с...","['Экономика', 'Газпром', 'Россия']",1299,2023-01-03 14:22:00,GAZP,163.13,163.37,1100961.10,1413064.30,0.24,0.147122,312103.2,28.348250
2,РИА,Экономика,https://ria.ru/20230104/shelf-1843063413.html,Ямальской шельфовой компании предоставили два ...,Правительство РФ предоставило Ямальской шель...,"['Экономика', 'Карское море', 'Россия']",1719,2023-01-04 16:36:00,ROSN,363.70,364.85,178263.05,321423.75,1.15,0.316195,143160.7,80.308679
3,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/879947,"""Соллерс"" возобновил после новогодних каникул ...","- Российская автомобилестроительная группа ""...","['Соллерс', 'Соллерс Алабуга', 'Татарстан']",2243,2023-01-09 12:28:00,SVAV,229.00,228.00,2290.00,4560.00,-1.00,-0.436681,2270.0,99.126638
4,Kommersant,Фондовый рынок,https://www.kommersant.ru/doc/5757640,Нестабильный рост,Минувший год запомнится инвесторам как время с...,NaN,12414,2023-01-06 10:02:00,SBER,141.31,141.40,1573757.40,2401025.00,0.09,0.063690,827267.6,52.566399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7537,РИА,Экономика,https://ria.ru/20231017/sberbank-1903333618.html,Сбербанк получил лицензию иностранного портфел...,Сбербанк России первым из отечественных банк...,"['Экономика', 'Индия', 'Россия', 'Сбербанк Рос...",1830,2023-10-17 14:39:00,SBER,271.26,270.79,33523733.50,17872073.00,-0.47,-0.173266,-15651660.5,-46.688298
7538,Интерфакс,В МИРЕ,https://www.interfax.ru/world/926287,Молдавия получит от ЕБРР еще 165 млн евро на з...,- Европейский банк реконструкции и развития ...,"['ЕБРР', 'Молдавия']",4682,2023-10-17 15:01:00,GAZP,172.04,171.67,2510079.70,47276358.20,-0.37,-0.215066,44766278.5,1783.460442
7539,РИА,Экономика,https://ria.ru/20231017/gaz-1903428772.html,Венгрия и Сербия пообещали Болгарии ответ на п...,Будапешт и Белград ответят на повышение плат...,"['Экономика', 'Россия', 'Венгрия', 'Сербия', '...",721,2023-10-17 18:13:00,GAZP,172.04,172.00,1367705.30,42397837.50,-0.04,-0.023250,41030132.2,2999.924925
7540,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/926333,FESCO и Liaoning Port будут развивать перевозк...,- Транспортная группа FESCO (головная компан...,"['FESCO', 'ВМТП', 'ДВМП', 'Подмосковье', 'Кита...",918,2023-10-17 18:21:00,FESH,98.07,98.70,9807.00,118440.00,0.63,0.642398,108633.0,1107.708779


### Выгрузка и проверка

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True
companies_df.to_sql('companies_trade', con=conn, if_exists='replace', index=False) 
conn.close()

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

dataframe = pd.read_sql_query('SELECT * FROM companies_trade', conn)
conn.close()

print(dataframe.shape)
dataframe.head()

# Индустриальные индексы

### Загрузка данных с базы данных

In [99]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

ind_df = pd.read_sql_query('SELECT * FROM industries_tickets', conn)
conn.close()

### Загрузка данных с Мосбиржи и обогащение существующего датасета

In [100]:
ind_string = 'http://iss.moex.com/iss/engines/stock/markets/index/boards/SNDX/securities/{}/candles.json?from={}&till={}&interval=1'

In [101]:
ind_df['price_release'] = np.nan
ind_df['volume_release'] = np.nan
ind_df['price_lag_30'] = np.nan
ind_df['volume_lag_30'] = np.nan

In [ ]:
ind_df = ind_df.sort_values(by=['datetime'])

In [121]:
ind_df['price_release'] = np.nan
ind_df['volume_release'] = np.nan
ind_df['price_lag_30'] = np.nan
ind_df['volume_lag_30'] = np.nan
for i in tqdm(range(len(ind_df))):
    start = ind_df['datetime'][i]
    end = ind_df['datetime'][i] + datetime.timedelta(minutes=30)
    ind = ind_df['industry'][i]
    connect_timeout = 100000
    read_timeout = 100000
    n = requests.get(ind_string.format(ind, start, end), timeout=(connect_timeout, read_timeout)).json()
    try:
        ind_df.loc[i, 'price_release'] = n['candles']['data'][0][0]
        ind_df.loc[i, 'volume_release'] = n['candles']['data'][0][4]
        ind_df.loc[i, 'price_lag_30'] = n['candles']['data'][-1][0]
        ind_df.loc[i, 'volume_lag_30'] = n['candles']['data'][-1][4]
    except IndexError:
        continue
    #delay = uniform(0, 3)
    #time.sleep(delay)

  7%|▋         | 412/6228 [08:44<2:03:26,  1.27s/it]


KeyboardInterrupt: 

In [189]:
ind_df.isna().sum()

website              0
section              0
url                  0
header               0
body                 0
tags               564
body_length          0
datetime             0
industry             0
price_release     1473
volume_release    1473
price_lag_30      1473
volume_lag_30     1473
dtype: int64

In [190]:
ind_df = ind_df.dropna(subset=['price_release'])

### Получение разниц

In [191]:
ind_df['price_diff'] = ind_df['price_lag_30'] - ind_df['price_release']
ind_df['price_diff_percent'] = ind_df['price_diff'] / ind_df['price_release'] * 100
ind_df['volume_diff'] = ind_df['volume_lag_30'] - ind_df['volume_release']
ind_df['volume_diff_percent'] = ind_df['volume_diff'] / ind_df['volume_release'] * 100

In [192]:
ind_df

,website,section,url,header,body,tags,body_length,datetime,industry,price_release,volume_release,price_lag_30,volume_lag_30,price_diff,price_diff_percent,volume_diff,volume_diff_percent
0,РИА,Экономика,https://ria.ru/20230103/banki-1842907991.html,Эксперт рассказал о развитии исламского банкин...,Порядка 10-15 филиалов исламского банкинга м...,"['Экономика', 'Россия', 'Сбербанк России']",1759,2023-01-03 10:16:00,MOEXFN,6307.20,1.680676e+07,6308.77,1.185410e+07,1.57,0.024886,-4.952657e+06,-41.780120
1,РИА,Экономика,https://ria.ru/20230103/gazprom-1842940413.html,"Экспорт ""Газпрома"" снизился до минимума с конц...","Объём экспорта ""Газпрома"" в прошедшем году с...","['Экономика', 'Газпром', 'Россия']",1299,2023-01-03 14:22:00,MOEXOG,5696.30,4.451181e+06,5696.72,4.605859e+06,0.42,0.007373,1.546786e+05,3.358301
2,РИА,Экономика,https://ria.ru/20230104/shelf-1843063413.html,Ямальской шельфовой компании предоставили два ...,Правительство РФ предоставило Ямальской шель...,"['Экономика', 'Карское море', 'Россия']",1719,2023-01-04 16:36:00,MOEXOG,5655.65,3.325798e+06,5663.65,1.490555e+06,8.00,0.141252,-1.835243e+06,-123.124780
3,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/879947,"""Соллерс"" возобновил после новогодних каникул ...","- Российская автомобилестроительная группа ""...","['Соллерс', 'Соллерс Алабуга', 'Татарстан']",2243,2023-01-09 12:28:00,MOEXCN,5377.67,2.791407e+05,5365.54,6.176750e+05,-12.13,-0.226072,3.385343e+05,54.807834
4,Kommersant,Фондовый рынок,https://www.kommersant.ru/doc/5757640,Нестабильный рост,Минувший год запомнится инвесторам как время с...,NaN,12414,2023-01-06 10:02:00,MOEXFN,6298.36,7.394623e+06,6296.27,3.371524e+06,-2.09,-0.033194,-4.023099e+06,-119.325840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/926309,"СД ""Ростелекома"" рекомендовал дивиденды-2022 в...","- Совет директоров ПАО ""Ростелеком"" рекоменд...",['Ростелеком'],2096,2023-10-17 16:35:00,MOEXFN,10388.73,1.414454e+07,10388.65,4.963418e+06,-0.08,-0.000770,-9.181126e+06,-184.975872
6223,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/926309,"СД ""Ростелекома"" рекомендовал дивиденды-2022 в...","- Совет директоров ПАО ""Ростелеком"" рекоменд...",['Ростелеком'],2096,2023-10-17 16:35:00,MOEXTL,2042.22,8.146566e+07,2041.61,2.950564e+07,-0.61,-0.029878,-5.196003e+07,-176.102025
6224,РИА,Экономика,https://ria.ru/20231017/eksport-1903397700.html,ГД приняла в I чтении проект об экспорте СПГ б...,Госдума приняла в первом чтении законопроект...,"['Экономика', 'Газпром', 'Новатэк', 'Госдума Р...",1997,2023-10-17 17:09:00,MOEXOG,9157.44,3.539749e+07,9156.21,1.986335e+07,-1.23,-0.013434,-1.553413e+07,-78.204988
6225,РИА,Экономика,https://ria.ru/20231017/gaz-1903428772.html,Венгрия и Сербия пообещали Болгарии ответ на п...,Будапешт и Белград ответят на повышение плат...,"['Экономика', 'Россия', 'Венгрия', 'Сербия', '...",721,2023-10-17 18:13:00,MOEXOG,9168.87,1.394318e+07,9169.17,1.687321e+07,0.30,0.003272,2.930027e+06,17.364963


### Выгрузка и проверка

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True
ind_df.to_sql('industries_trade', con=conn, if_exists='replace', index=False) 
conn.close()

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

dataframe = pd.read_sql_query('SELECT * FROM industries_trade', conn)
conn.close()

print(dataframe.shape)
dataframe.head()

# Индекс Мосбиржи, RVI и фьючерс на курс доллара

### Загрузка данных с базы данных

In [134]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

global_df = pd.read_sql_query('SELECT * FROM global_tickets', conn)
conn.close()

### Загрузка данных с Мосбиржи и обогащение существующего датасета

In [135]:
global_moex = 'http://iss.moex.com/iss/engines/stock/markets/index/boards/SNDX/securities/IMOEX2/candles.json?from={}&till={}&interval=1'
global_rvi = 'http://iss.moex.com/iss/engines/stock/markets/index/boards/RTSI/securities/RVI/candles.json?from={}&till={}&interval=1'
global_usd_rub_fut = 'http://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/{}/candles.json?from={}&till={}&interval=1'

In [142]:
global_df['imoex_price_release'] = np.nan
global_df['imoex_volume_release'] = np.nan
global_df['imoex_price_lag_30'] = np.nan
global_df['imoex_volume_lag_30'] = np.nan
global_df['rvi_price_release'] = np.nan
global_df['rvi_price_lag_30'] = np.nan
global_df['usd_price_release'] = np.nan
global_df['usd_price_lag_30'] = np.nan

In [145]:
global_df = global_df.sort_values(by=['datetime'])

In [158]:
for i in tqdm(range(len(global_df))):
    start = global_df['datetime'][i]
    end = global_df['datetime'][i] + datetime.timedelta(minutes=30)
    connect_timeout = 300
    read_timeout = 120
    n = requests.get(global_moex.format(start, end), timeout=(connect_timeout, read_timeout)).json()
    try:
        global_df.loc[i, 'imoex_price_release'] = n['candles']['data'][0][0]
        global_df.loc[i, 'imoex_volume_release'] = n['candles']['data'][0][4]
        global_df.loc[i, 'imoex_price_lag_30'] = n['candles']['data'][-1][0]
        global_df.loc[i, 'imoex_volume_lag_30'] = n['candles']['data'][-1][4]
    except IndexError:
        continue
    n = requests.get(global_rvi.format(start, end), timeout=(connect_timeout, read_timeout)).json()
    try:
        global_df.loc[i, 'rvi_price_release'] = n['candles']['data'][0][0]
        global_df.loc[i, 'rvi_price_lag_30'] = n['candles']['data'][-1][0]
    except IndexError:
        continue
    if str(start) < '2023-03-16':
        n = requests.get(global_usd_rub_fut.format('SiH3', start, end), timeout=(connect_timeout, read_timeout)).json()
        try:
            global_df.loc[i, 'usd_price_release'] = n['candles']['data'][0][0]
            global_df.loc[i, 'usd_price_lag_30'] = n['candles']['data'][-1][0]
        except IndexError:
            continue
    elif str(start) < '2023-06-15':
        n = requests.get(global_usd_rub_fut.format('SiM3', start, end), timeout=(connect_timeout, read_timeout)).json()
        try:
            global_df.loc[i, 'usd_price_release'] = n['candles']['data'][0][0]
            global_df.loc[i, 'usd_price_lag_30'] = n['candles']['data'][-1][0]
        except IndexError:
            continue
    elif str(start) < '2023-09-21':
        n = requests.get(global_usd_rub_fut.format('SiU3', start, end), timeout=(connect_timeout, read_timeout)).json()
        try:
            global_df.loc[i, 'usd_price_release'] = n['candles']['data'][0][0]
            global_df.loc[i, 'usd_price_lag_30'] = n['candles']['data'][-1][0]
        except IndexError:
            continue
    else:
        n = requests.get(global_usd_rub_fut.format('SiZ3', start, end), timeout=(connect_timeout, read_timeout)).json()
        try:
            global_df.loc[i, 'usd_price_release'] = n['candles']['data'][0][0]
            global_df.loc[i, 'usd_price_lag_30'] = n['candles']['data'][-1][0]
        except IndexError:
            continue
    #delay = randint(1,2)
    #time.sleep(delay)

  0%|          | 30/13326 [03:58<29:22:12,  7.95s/it]


ConnectTimeout: HTTPConnectionPool(host='iss.moex.com', port=80): Max retries exceeded with url: /iss/engines/stock/markets/index/boards/RTSI/securities/RVI/candles.json?from=2023-01-04%2021:57:00&till=2023-01-04%2022:27:00&interval=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001ECB8EFB190>, 'Connection to iss.moex.com timed out. (connect timeout=300)'))

In [ ]:
global_df.isna().sum()

In [ ]:
global_df = global_df.dropna(subset=['price_release'])

### Получение разниц

In [ ]:
global_df['imoex_price_diff'] = global_df['imoex_price_lag_30'] - global_df['imoex_price_release']
global_df['imoex_price_diff_percent'] = global_df['imoex_price_diff'] / global_df['imoex_price_release'] * 100
global_df['imoex_volume_diff'] = global_df['imoex_volume_lag_30'] - global_df['imoex_volume_release']
global_df['imoex_volume_diff_percent'] = global_df['imoex_volume_diff'] / global_df['imoex_volume_release'] * 100
global_df['rvi_price_diff'] = global_df['rvi_price_lag_30'] - global_df['rvi_price_release']
global_df['rvi_price_diff_percent'] = global_df['rvi_price_diff'] / global_df['rvi_price_release'] * 100
global_df['usd_price_diff'] = global_df['usd_price_lag_30'] - global_df['usd_price_release']
global_df['usd_price_diff_percent'] = global_df['usd_price_diff'] / global_df['usd_price_release'] * 100

In [ ]:
global_df

### Выгрузка и проверка

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True
global_df.to_sql('global_trade', con=conn, if_exists='replace', index=False) 
conn.close()

In [ ]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

dataframe = pd.read_sql_query('SELECT * FROM global_trade', conn)
conn.close()

print(dataframe.shape)
dataframe.head()